In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pickle
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
import xgboost

In [2]:
crop_prod=pd.read_csv('crop_production.csv')
crop_prod.head()

,State_Name,District_Name,Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [3]:
#dropping data from 2002
crop_prod.drop(crop_prod[crop_prod['Year']>2002].index,inplace=True)

In [4]:
tempdata=pd.read_excel('Final.xlsx')
tempdata.head()

,District_Name,Year,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,Max Temp Kharif,Max Temp Whole,Vapour Press Rabi,Vapour Press Kharif,Vapour Press Whole,RCE Rabi,RCE Kharif,RCE Whole,Precipitation Rabi,Precipitation Kharif,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole
0,BALAGHAT,1990,17.370667,22.952833,20.161750,32.308000,32.832667,32.570333,13.641833,22.434833,18.038333,4.641667,5.103333,4.872500,12.074833,243.787833,127.931333,7.7878,62.4487,70.2365
1,BALAGHAT,1991,16.665667,24.503333,20.584500,32.090500,34.523000,33.306750,13.689333,24.056000,18.872667,4.681667,5.361667,5.021667,12.324833,194.985167,103.655000,8.5927,46.9924,55.5851
2,BALAGHAT,1992,16.941667,24.739667,20.840667,32.363833,34.751500,33.557667,13.791500,24.122333,18.956917,4.713333,5.378333,5.045833,4.657167,192.645500,98.651333,4.5710,49.5393,54.1103
3,BALAGHAT,1993,16.311833,24.607333,20.459583,31.727000,34.614167,33.170583,13.393167,23.912500,18.652833,4.630000,5.365000,4.997500,14.876000,226.780500,120.828250,7.9172,52.8459,60.7631
4,BALAGHAT,1994,16.680667,23.800333,20.240500,32.101667,33.825000,32.963333,13.638333,23.330500,18.484417,4.681667,5.283333,4.982500,23.894167,318.656000,171.275083,8.8731,63.5572,72.4303


In [5]:
data=pd.merge(crop_prod,tempdata,'inner',on=['District_Name','Year'])
data.drop('Year',axis=1,inplace=True)
data.head()

,State_Name,District_Name,Season,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,...,Vapour Press Whole,RCE Rabi,RCE Kharif,RCE Whole,Precipitation Rabi,Precipitation Kharif,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole
0,Himachal Pradesh,UNA,Kharif,Groundnut,3.0,3.0,10.410833,24.737,17.573917,25.2665,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
1,Himachal Pradesh,UNA,Kharif,Horse-gram,57.0,20.0,10.410833,24.737,17.573917,25.2665,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
2,Himachal Pradesh,UNA,Kharif,Maize,28084.0,52574.0,10.410833,24.737,17.573917,25.2665,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
3,Himachal Pradesh,UNA,Kharif,Moong(Green Gram),34.0,16.0,10.410833,24.737,17.573917,25.2665,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
4,Himachal Pradesh,UNA,Kharif,Potato,89.0,1048.0,10.410833,24.737,17.573917,25.2665,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092


In [6]:
data.shape
columns=data.columns
print(columns)

Index(['State_Name', 'District_Name', 'Season', 'Crop', 'Area', 'Production',
       'Min Temp Rabi', 'Min Temp Kharif', 'Min Temp Whole', 'Max Temp Rabi',
       'Max Temp Kharif', 'Max Temp Whole ', 'Vapour Press Rabi',
       'Vapour Press Kharif', 'Vapour Press Whole', 'RCE Rabi', 'RCE Kharif',
       'RCE  Whole', 'Precipitation Rabi', 'Precipitation Kharif',
       'Precipitaion Whole', 'Wet Day Freq Rabi', 'Wet Day Freq Kharif',
       'Wet Day Freq Whole'],
      dtype='object')


In [7]:
Min_temp=np.zeros((3072,6))
Min_temp=pd.DataFrame(Min_temp)
Min_temp.columns=['Min Temp','Max Temp','Vapour Pressure','RCE','Precipitation','Wet Day']

temp=pd.concat([data,Min_temp],axis=1)
columns=temp.columns

In [8]:
temp.tail()

,State_Name,District_Name,Season,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,...,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
3067,West Bengal,JALPAIGURI,Whole Year,Tobacco,494.0,405.0,15.227333,24.24,19.733667,28.5295,...,180.113,11.7928,71.2522,83.045,0.0,0.0,0.0,0.0,0.0,0.0
3068,West Bengal,JALPAIGURI,Whole Year,Turmeric,1525.0,1865.0,15.227333,24.24,19.733667,28.5295,...,180.113,11.7928,71.2522,83.045,0.0,0.0,0.0,0.0,0.0,0.0
3069,West Bengal,JALPAIGURI,Winter,Potato,24594.0,483185.0,15.227333,24.24,19.733667,28.5295,...,180.113,11.7928,71.2522,83.045,0.0,0.0,0.0,0.0,0.0,0.0
3070,West Bengal,JALPAIGURI,Winter,Rice,188385.0,298543.0,15.227333,24.24,19.733667,28.5295,...,180.113,11.7928,71.2522,83.045,0.0,0.0,0.0,0.0,0.0,0.0
3071,West Bengal,JALPAIGURI,Winter,Sesamum,80.0,48.0,15.227333,24.24,19.733667,28.5295,...,180.113,11.7928,71.2522,83.045,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(columns)

Index(['State_Name', 'District_Name', 'Season', 'Crop', 'Area', 'Production',
       'Min Temp Rabi', 'Min Temp Kharif', 'Min Temp Whole', 'Max Temp Rabi',
       'Max Temp Kharif', 'Max Temp Whole ', 'Vapour Press Rabi',
       'Vapour Press Kharif', 'Vapour Press Whole', 'RCE Rabi', 'RCE Kharif',
       'RCE  Whole', 'Precipitation Rabi', 'Precipitation Kharif',
       'Precipitaion Whole', 'Wet Day Freq Rabi', 'Wet Day Freq Kharif',
       'Wet Day Freq Whole', 'Min Temp', 'Max Temp', 'Vapour Pressure', 'RCE',
       'Precipitation', 'Wet Day'],
      dtype='object')


In [10]:
temp['Season'].unique()
# temp[temp['Season']=='Summer     ']
# temp.columns

array(['Kharif     ', 'Rabi       ', 'Whole Year ', 'Summer     ',
       'Autumn     ', 'Winter     '], dtype=object)

In [11]:
for i in range(3071):
    if(temp['Season'].iloc[i]=='Rabi       ' or temp['Season'].iloc[i]=='Winter     '):
        temp['Min Temp'].iloc[i]=temp['Min Temp Rabi'].iloc[i]
        temp['Max Temp'].iloc[i]=temp['Max Temp Rabi'].iloc[i]
        temp['Vapour Pressure'].iloc[i]=temp['Vapour Press Rabi'].iloc[i]
        temp['RCE'].iloc[i]=temp['RCE Rabi'].iloc[i]
        temp['Precipitation'].iloc[i]=temp['Precipitation Rabi'].iloc[i]
        temp['Wet Day'].iloc[i]=temp['Wet Day Freq Rabi'].iloc[i]
    elif(temp['Season'].iloc[i]=='Kharif     ' or temp['Season'].iloc[i]=='Autumn     ' ):
        temp['Min Temp'].iloc[i]=temp['Min Temp Kharif'].iloc[i]
        temp['Max Temp'].iloc[i]=temp['Max Temp Kharif'].iloc[i]
        temp['Vapour Pressure'].iloc[i]=temp['Vapour Press Kharif'].iloc[i]
        temp['RCE'].iloc[i]=temp['RCE Kharif'].iloc[i]
        temp['Precipitation'].iloc[i]=temp['Precipitation Kharif'].iloc[i]
        temp['Wet Day'].iloc[i]=temp['Wet Day Freq Kharif'].iloc[i]
    elif(temp['Season'].iloc[i]=='Whole Year '  ):
        temp['Min Temp'].iloc[i]=temp['Min Temp Whole'].iloc[i]
        temp['Max Temp'].iloc[i]=temp['Max Temp Rabi'].iloc[i]
        temp['Vapour Pressure'].iloc[i]=temp['Vapour Press Whole'].iloc[i]
        temp['RCE'].iloc[i]=temp['RCE  Whole'].iloc[i]
        temp['Precipitation'].iloc[i]=temp['Precipitaion Whole'].iloc[i]
        temp['Wet Day'].iloc[i]=temp['Wet Day Freq Whole'].iloc[i]
    elif(temp['Season'].iloc[i]=='Summer     '):
        temp['Min Temp'].iloc[i]=temp['Min Temp Kharif'].iloc[i]
        temp['Max Temp'].iloc[i]=temp['Max Temp Kharif'].iloc[i]
        temp['Vapour Pressure'].iloc[i]=temp['Vapour Press Rabi'].iloc[i]
        temp['RCE'].iloc[i]=temp['RCE  Whole'].iloc[i]
        temp['Precipitation'].iloc[i]=temp['Precipitation Rabi'].iloc[i]
        temp['Wet Day'].iloc[i]=temp['Wet Day Freq Rabi'].iloc[i]
        
    


C:\Users\Shubham\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
temp[temp['Season']=='Winter     ']

,State_Name,District_Name,Season,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,...,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
2909,West Bengal,JALPAIGURI,Winter,Rice,178783.0,233560.0,14.788667,23.966667,19.377667,27.528500,...,173.974917,11.4231,73.4783,84.9014,14.788667,27.528500,17.127833,3.660000,26.712167,11.4231
2939,West Bengal,JALPAIGURI,Winter,Rice,176434.0,191660.0,15.652833,25.019167,20.336000,27.921000,...,199.576417,10.7747,72.0741,82.8488,15.652833,27.921000,17.946167,3.610000,26.380000,10.7747
2971,West Bengal,JALPAIGURI,Winter,Potato,14490.0,329191.0,16.336667,24.422167,20.379417,29.528167,...,198.417917,6.8061,76.5057,83.3118,16.336667,29.528167,17.977500,3.910000,4.076833,6.8061
2972,West Bengal,JALPAIGURI,Winter,Rice,188551.0,261620.0,16.336667,24.422167,20.379417,29.528167,...,198.417917,6.8061,76.5057,83.3118,16.336667,29.528167,17.977500,3.910000,4.076833,6.8061
2973,West Bengal,JALPAIGURI,Winter,Sesamum,25.0,12.0,16.336667,24.422167,20.379417,29.528167,...,198.417917,6.8061,76.5057,83.3118,16.336667,29.528167,17.977500,3.910000,4.076833,6.8061
3002,West Bengal,JALPAIGURI,Winter,Rice,190854.0,269210.0,15.099500,24.517833,19.808667,27.714000,...,172.384417,7.3714,74.2597,81.6311,15.099500,27.714000,17.441000,3.625000,14.187167,7.3714
3034,West Bengal,JALPAIGURI,Winter,Rice,176149.0,274047.0,15.534333,25.056000,20.295167,28.443167,...,168.217833,9.3938,71.7292,81.1230,15.534333,28.443167,17.636500,3.721667,18.743667,9.3938
3069,West Bengal,JALPAIGURI,Winter,Potato,24594.0,483185.0,15.227333,24.240000,19.733667,28.529500,...,180.113000,11.7928,71.2522,83.0450,15.227333,28.529500,17.402500,3.785000,27.519167,11.7928
3070,West Bengal,JALPAIGURI,Winter,Rice,188385.0,298543.0,15.227333,24.240000,19.733667,28.529500,...,180.113000,11.7928,71.2522,83.0450,15.227333,28.529500,17.402500,3.785000,27.519167,11.7928
3071,West Bengal,JALPAIGURI,Winter,Sesamum,80.0,48.0,15.227333,24.240000,19.733667,28.529500,...,180.113000,11.7928,71.2522,83.0450,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000


In [13]:
temp.head()

,State_Name,District_Name,Season,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,...,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,Himachal Pradesh,UNA,Kharif,Groundnut,3.0,3.0,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,Himachal Pradesh,UNA,Kharif,Horse-gram,57.0,20.0,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,Himachal Pradesh,UNA,Kharif,Maize,28084.0,52574.0,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,Himachal Pradesh,UNA,Kharif,Moong(Green Gram),34.0,16.0,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,Himachal Pradesh,UNA,Kharif,Potato,89.0,1048.0,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [14]:
#removing state name ,Distric name and season
data=data.drop(['State_Name','District_Name','Season'],axis=1)
data.head()


,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,Max Temp Kharif,Max Temp Whole,Vapour Press Rabi,...,Vapour Press Whole,RCE Rabi,RCE Kharif,RCE Whole,Precipitation Rabi,Precipitation Kharif,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole
0,Groundnut,3.0,3.0,10.410833,24.737,17.573917,25.2665,36.409333,30.837917,11.465833,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
1,Horse-gram,57.0,20.0,10.410833,24.737,17.573917,25.2665,36.409333,30.837917,11.465833,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
2,Maize,28084.0,52574.0,10.410833,24.737,17.573917,25.2665,36.409333,30.837917,11.465833,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
3,Moong(Green Gram),34.0,16.0,10.410833,24.737,17.573917,25.2665,36.409333,30.837917,11.465833,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092
4,Potato,89.0,1048.0,10.410833,24.737,17.573917,25.2665,36.409333,30.837917,11.465833,...,17.9145,3.381667,5.803333,4.5925,24.607,85.51,55.0585,11.9275,20.9817,32.9092


In [15]:
data.shape

(3072, 21)

In [16]:
temp['Production']=data.iloc[:,2]/data.iloc[:,1]
temp['Area']=data.iloc[:,1]/data.iloc[:,1]
temp.head()

,State_Name,District_Name,Season,Crop,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,...,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,Himachal Pradesh,UNA,Kharif,Groundnut,1.0,1.000000,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,Himachal Pradesh,UNA,Kharif,Horse-gram,1.0,0.350877,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,Himachal Pradesh,UNA,Kharif,Maize,1.0,1.872027,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,Himachal Pradesh,UNA,Kharif,Moong(Green Gram),1.0,0.470588,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,Himachal Pradesh,UNA,Kharif,Potato,1.0,11.775281,10.410833,24.737,17.573917,25.2665,...,55.0585,11.9275,20.9817,32.9092,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [17]:
columns

Index(['State_Name', 'District_Name', 'Season', 'Crop', 'Area', 'Production',
       'Min Temp Rabi', 'Min Temp Kharif', 'Min Temp Whole', 'Max Temp Rabi',
       'Max Temp Kharif', 'Max Temp Whole ', 'Vapour Press Rabi',
       'Vapour Press Kharif', 'Vapour Press Whole', 'RCE Rabi', 'RCE Kharif',
       'RCE  Whole', 'Precipitation Rabi', 'Precipitation Kharif',
       'Precipitaion Whole', 'Wet Day Freq Rabi', 'Wet Day Freq Kharif',
       'Wet Day Freq Whole', 'Min Temp', 'Max Temp', 'Vapour Pressure', 'RCE',
       'Precipitation', 'Wet Day'],
      dtype='object')

In [18]:
temp=temp.drop(columns[6:-6],axis=1)

In [19]:
temp=temp.drop(['State_Name','District_Name','Season'],axis=1)
temp.head()

,Crop,Area,Production,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,Groundnut,1.0,1.000000,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,Horse-gram,1.0,0.350877,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,Maize,1.0,1.872027,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,Moong(Green Gram),1.0,0.470588,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,Potato,1.0,11.775281,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [20]:
# colindex=[1]
# for i in range(3,21):
#     colindex.append(i)
# scaler=StandardScaler()
# tempdata=scaler.fit_transform(temp.iloc[:,3:])


In [21]:
#temp.iloc[:,3:]=tempdata

In [22]:
temp.head()

,Crop,Area,Production,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,Groundnut,1.0,1.000000,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,Horse-gram,1.0,0.350877,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,Maize,1.0,1.872027,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,Moong(Green Gram),1.0,0.470588,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,Potato,1.0,11.775281,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [23]:
#Encoding of Crops into numerical features
#temp=data.copy()
#temp=temp[temp['Crop']=='Groundnut' or temp['Crop']=='Maize'  or temp['Crop']=='Potato' or temp['Crop']=='Horse-gram' ]
# enc=LabelEncoder()
# enc_test=enc.fit_transform(temp['Crop'])
# temp['Crop']=enc_test
temp.head()

,Crop,Area,Production,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,Groundnut,1.0,1.000000,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,Horse-gram,1.0,0.350877,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,Maize,1.0,1.872027,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,Moong(Green Gram),1.0,0.470588,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,Potato,1.0,11.775281,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [26]:
forcor=temp.iloc[:,2:]
forcor.corr()

,Production,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
Production,1.000000,0.003582,-0.050154,0.031648,-0.038443,0.019440,0.096437
Min Temp,0.003582,1.000000,0.776787,0.830656,0.713224,0.528312,0.451382
Max Temp,-0.050154,0.776787,1.000000,0.455796,0.881495,0.112287,-0.005064
Vapour Pressure,0.031648,0.830656,0.455796,1.000000,0.366720,0.757177,0.697904
RCE,-0.038443,0.713224,0.881495,0.366720,1.000000,0.022814,-0.056693
Precipitation,0.019440,0.528312,0.112287,0.757177,0.022814,1.000000,0.840462
Wet Day,0.096437,0.451382,-0.005064,0.697904,-0.056693,0.840462,1.000000


In [27]:
data.head()
forcor=data.iloc[:,1:]
forcor.corr()

,Area,Production,Min Temp Rabi,Min Temp Kharif,Min Temp Whole,Max Temp Rabi,Max Temp Kharif,Max Temp Whole,Vapour Press Rabi,Vapour Press Kharif,Vapour Press Whole,RCE Rabi,RCE Kharif,RCE Whole,Precipitation Rabi,Precipitation Kharif,Precipitaion Whole,Wet Day Freq Rabi,Wet Day Freq Kharif,Wet Day Freq Whole
Area,1.000000,0.254092,-0.089556,0.034758,-0.096771,-0.089372,0.064132,-0.010490,-0.055289,-0.054944,-0.058502,-0.080123,0.065964,0.014788,-0.037242,-0.038059,-0.041398,-0.021972,-0.039602,-0.040482
Production,0.254092,1.000000,0.015069,-0.015628,0.012528,-0.032023,-0.028128,-0.060750,0.040328,0.013444,0.033392,-0.040184,-0.032576,-0.055940,0.042303,0.011683,0.018700,0.035399,0.020925,0.026973
Min Temp Rabi,-0.089556,0.015069,1.000000,-0.698172,0.961394,0.717128,-0.879932,-0.322788,0.877181,0.467270,0.787014,0.532703,-0.866621,-0.511228,0.643459,0.639822,0.699316,0.526239,0.733638,0.777775
Min Temp Kharif,0.034758,-0.015628,-0.698172,1.000000,-0.474211,-0.604224,0.826078,0.366300,-0.526938,-0.029222,-0.385004,-0.554772,0.696262,0.333445,-0.321755,-0.469104,-0.483728,-0.192293,-0.596496,-0.577383
Min Temp Whole,-0.096771,0.012528,0.961394,-0.474211,1.000000,0.649644,-0.764579,-0.256155,0.876173,0.563391,0.819845,0.441856,-0.798106,-0.500517,0.667620,0.606512,0.674054,0.573229,0.672916,0.734540
Max Temp Rabi,-0.089372,-0.032023,0.717128,-0.604224,0.649644,1.000000,-0.529427,0.326979,0.364073,0.009262,0.262193,0.961113,-0.433704,0.167519,0.111024,0.314445,0.304020,-0.050735,0.451799,0.391472
Max Temp Kharif,0.064132,-0.028128,-0.879932,0.826078,-0.764579,-0.529427,1.000000,0.628611,-0.843797,-0.514211,-0.779652,-0.380692,0.978837,0.712136,-0.552573,-0.775739,-0.803884,-0.444427,-0.865345,-0.876248
Max Temp Whole,-0.010490,-0.060750,-0.322788,0.366300,-0.256155,0.326979,0.628611,1.000000,-0.606195,-0.564318,-0.628136,0.457014,0.692790,0.946859,-0.513763,-0.575876,-0.616786,-0.541584,-0.549776,-0.617226
Vapour Press Rabi,-0.055289,0.040328,0.877181,-0.526938,0.876173,0.364073,-0.843797,-0.606195,1.000000,0.749755,0.972966,0.151582,-0.892373,-0.768393,0.753582,0.732173,0.803588,0.722087,0.774056,0.859581
Vapour Press Kharif,-0.054944,0.013444,0.467270,-0.029222,0.563391,0.009262,-0.514211,-0.564318,0.749755,1.000000,0.882309,-0.165603,-0.647118,-0.725166,0.522233,0.694284,0.724825,0.578619,0.639120,0.705629


In [25]:
#Dropping NaN Values 
temp=temp[temp['Production'].notna()]
data=data[data['Production'].notna()]

In [26]:
# temp=temp.drop('Production',axis=1)
labelenc=LabelEncoder()
temp['Crop']=labelenc.fit_transform(temp['Crop'])

In [27]:
# category=[0]
# category=np.reshape(category,(1,-1))
# ct=ColumnTransformer([('Crop',OneHotEncoder(),[0])],remainder='passthrough')
# #onehotencoder=OneHotEncoder(categories='auto')
# temp=ct.fit_transform(temp)
temp.head()

,Crop,Area,Production,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,31,1.0,1.000000,24.737,36.409333,24.363167,5.803333,85.51,20.9817
1,33,1.0,0.350877,24.737,36.409333,24.363167,5.803333,85.51,20.9817
2,41,1.0,1.872027,24.737,36.409333,24.363167,5.803333,85.51,20.9817
3,45,1.0,0.470588,24.737,36.409333,24.363167,5.803333,85.51,20.9817
4,63,1.0,11.775281,24.737,36.409333,24.363167,5.803333,85.51,20.9817


In [28]:
temp.shape

(3036, 9)

In [29]:
colindex=[0]
for i in range(3,9):
    colindex.append(i)
colindex

[0, 3, 4, 5, 6, 7, 8]

In [30]:
traindata=temp.iloc[:,3:]
testdata=temp.iloc[:,2]
traindata

,Min Temp,Max Temp,Vapour Pressure,RCE,Precipitation,Wet Day
0,24.737000,36.409333,24.363167,5.803333,85.510000,20.9817
1,24.737000,36.409333,24.363167,5.803333,85.510000,20.9817
2,24.737000,36.409333,24.363167,5.803333,85.510000,20.9817
3,24.737000,36.409333,24.363167,5.803333,85.510000,20.9817
4,24.737000,36.409333,24.363167,5.803333,85.510000,20.9817
...,...,...,...,...,...,...
3067,19.733667,28.529500,23.938667,4.133333,180.113000,83.0450
3068,19.733667,28.529500,23.938667,4.133333,180.113000,83.0450
3069,15.227333,28.529500,17.402500,3.785000,27.519167,11.7928
3070,15.227333,28.529500,17.402500,3.785000,27.519167,11.7928


In [31]:
X_train,X_test,y_train,y_test=train_test_split(traindata,testdata,test_size=0.1)

In [36]:
lasso=RandomForestRegressor(n_estimators=1000,max_depth=10000,min_samples_leaf=8,min_samples_split=8)
lasso.fit(X_train,y_train)


RandomForestRegressor(max_depth=10000, min_samples_leaf=8, min_samples_split=8,
                      n_estimators=1000)

In [37]:
lassores=lasso.predict(X_test)
print(lassores[0:5])
print(y_test[0:5])
print(X_test)

[ 0.84587576  1.15548857 26.11445154  2.03534366  1.53779881]
812     1.035294
367     1.488889
33      0.428571
1107    4.179916
1607    0.543901
Name: Production, dtype: float64
       Min Temp   Max Temp  Vapour Pressure       RCE  Precipitation  Wet Day
812   22.956333  34.073167        22.279500  5.496667     115.767333  39.1288
367   24.856667  34.719000        24.235167  5.340000     174.867833  50.0154
33    18.615083  27.109000        18.637667  4.708333      43.618000  28.1879
1107  12.487833  28.029500        10.976667  3.763333       4.786500   6.7090
1607  12.198500  27.978500         7.516000  3.953333      10.203667   6.3257
...         ...        ...              ...       ...            ...      ...
1338  14.408000  30.120333         7.913833  4.208333       1.955000   3.6699
1618  25.113667  36.319000        21.717167  5.746667      94.658833  26.6487
866   19.817583  33.734667        17.258750  5.275000      72.515667  45.7444
1690  13.921000  29.668167         8.234

In [34]:
#calculating error
errorf=mean_squared_error(y_test,lassores)
print(np.sqrt(errorf) )

13.828315575565089


In [35]:
print('R2: ',r2_score(y_test,lassores))
#print('oob',regf.oob_score_)


R2:  -0.7850190471963638


svr=SVR(C=200000,kernel='poly',degree=3,)
svr.fit(X_train,y_train)

In [273]:
svrres=svr.predict(X_test)
#calculating error
errorsvr=mean_squared_error(y_test,svrres)
print(np.sqrt(errorf) )
print('R2: ',r2_score(y_test,svrres))
#print('oob',regf.oob_score_)


5580.477874710055
R2:  0.8825461771035296


In [323]:
arr=[300,19,25.046333,19.296500,28.879000,36.329667,32.604333,8.790833,22.593000,15.691917,3.990000,5.751667,4.870833,5.053667,42.508500,23.781083,6.3200,19.1520,25.472]
arr=scaler.transform(np.array(arr).reshape(1,-1))
a=traindata.iloc[0,:]
a=np.array(a).reshape(1,-1)
a

array([[-0.33872943, -1.58327626,  0.62239327, -1.70780099, -2.26365605,
         0.96464538, -1.00060572, -0.38358083, -0.00322753, -0.27396329,
        -1.97662026,  0.93264352, -0.30526744,  0.46309398, -0.67864118,
        -0.52031914,  0.84515293, -1.00405918, -0.69899408]])

In [324]:
lasso.predict(arr)

array([12068.5691307])

In [293]:
xgreg=xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
xgreg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.07, max_delta_step=0, max_depth=3,
             min_child_weight=1.5, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             subsample=0.6, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [295]:
xgboost

ValueError: feature_names mismatch: ['Area', 'Min Temp Rabi', 'Min Temp Kharif', 'Min Temp Whole', 'Max Temp Rabi', 'Max Temp Kharif', 'Max Temp Whole ', 'Vapour Press Rabi', 'Vapour Press Kharif', 'Vapour Press Whole', 'RCE Rabi', 'RCE Kharif', 'RCE  Whole', 'Precipitation Rabi', 'Precipitation Kharif', 'Precipitaion Whole', 'Wet Day Freq Rabi', 'Wet Day Freq Kharif', 'Wet Day Freq Whole'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18']
expected Max Temp Kharif, Max Temp Whole , Min Temp Whole, Precipitation Kharif, Min Temp Rabi, Vapour Press Whole, Wet Day Freq Kharif, RCE  Whole, Wet Day Freq Rabi, Vapour Press Kharif, Max Temp Rabi, Precipitaion Whole, Precipitation Rabi, Area, Vapour Press Rabi, Min Temp Kharif, RCE Kharif, RCE Rabi, Wet Day Freq Whole in input data
training data did not have the following fields: f6, f15, f0, f12, f4, f10, f8, f2, f17, f9, f1, f7, f14, f11, f5, f16, f13, f3, f18